In [4]:
import matplotlib
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['font.family'] = 'serf'
import matplotlib.pyplot as plt
import numpy as np

In [5]:
def solve_ftcs(f, v, dx, dt):
    f_next = np.zeros(len(f))
    f_next = f - v * dt / 2.0 / dx * (np.roll(f,-1) - np.roll(f, 1))
    return f_next

def solve_lax(f, v, dx, dt):
    f_next = np.zeros(len(f))
    f_next = 0.5 * (np.roll(f, 1) + np.roll(f, -1)) - v * dt / 2.0 / dx * (np.roll(f,-1) - np.roll(f, 1))
    return f_next

def solve_upwind(f, v, dx, dt):
    # f_next = np.zeros(len(f))
    f_next = f - v * dt / dx * (f - np.roll(f, 1))
    return f_next

def solve_leapfrog(f, f1, v, dx, dt):
    f_next = np.zeros(len(f))
    f_next = f1 - v * dt / dx * (np.roll(f,-1) - np.roll(f, 1))
    return f_next

def solve_lax_wendroff(f, v, dx, dt):
    f1 = 0.5 * (np.roll(f, 1) + f) - v * dt / 2.0 / dx * (f - np.roll(f, 1))
    f2 = 0.5 * (f + np.roll(f, -1)) - v * dt / 2.0 / dx * (np.roll(f, -1) - f)
    f_next = f - v * dt / dx * (f2 - f1)
    return f_next

In [34]:
def solve_lax_2d(f, vx, vy, dx, dy, dt):
    f_next = 0.25 * (np.roll(f, 1, axis=0) + np.roll(f, -1, axis=0) +
                     np.roll(f, 1, axis=1) + np.roll(f, -1, axis=1)) - \
              vx * dt / 2.0 / dx * (np.roll(f,-1, axis=1) - np.roll(f, 1, axis=1)) - \
              vy * dt / 2.0 / dy * (np.roll(f,-1, axis=0) - np.roll(f, 1, axis=0))
    return f_next

def solve_upwind_2d(f, vx, vy, dx, dy, dt):
    f_next = f - vx * dt / dx * (f - np.roll(f, 1, axis=1)) - vy * dt / dy * (f - np.roll(f, 1, axis=0))
    return f_next

def solve_lax_wendroff_2d(f, vx, vy, dx, dy, dt):
    f_avg_x = 0.5 * (np.roll(f, 0, axis=1) + np.roll(f, -1, axis=1))
    f_avg_y = 0.5 * (np.roll(f, 0, axis=0) + np.roll(f, -1, axis=0))
    f_half = 0.25 * (np.roll(f, 0, axis=0) + np.roll(f, -1, axis=0) +
                     np.roll(f, 0, axis=1) + np.roll(f, -1, axis=1)) - \
              vx * dt / 2.0 / dx * (np.roll(f_avg_y,-1, axis=1) - np.roll(f_avg_y, 0, axis=1)) - \
              vy * dt / 2.0 / dy * (np.roll(f_avg_x,-1, axis=0) - np.roll(f_avg_x, 0, axis=0))
    f_half_avg_x = 0.5 * (np.roll(f_half, 1, axis=1) + np.roll(f_half, 0, axis=1))
    f_half_avg_y = 0.5 * (np.roll(f_half, 1, axis=0) + np.roll(f_half, 0, axis=0))
    f_next = f - vx * dt / dx * (f_half - np.roll(f_half, 1, axis=0)) - vy * dt / dy * (f_half - np.roll(f_half, 1, axis=1))
    return f_next

def solve_fcts_diffusion(f, D, dx, dt):
    f_next = np.zeros(len(f))
    f_next = f + D * dt / dx**2 * (np.roll(f,-1) - 2 * f + np.roll(f, 1))
    return f_next

In [76]:
print(np.roll(y, -1, axis=0))

[[0.00787402 0.00787402 0.00787402 ... 0.00787402 0.00787402 0.00787402]
 [0.01574803 0.01574803 0.01574803 ... 0.01574803 0.01574803 0.01574803]
 [0.02362205 0.02362205 0.02362205 ... 0.02362205 0.02362205 0.02362205]
 ...
 [0.99212598 0.99212598 0.99212598 ... 0.99212598 0.99212598 0.99212598]
 [1.         1.         1.         ... 1.         1.         1.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [35]:
nx = 128
ny = 128
lx = 1.0
ly = 1.0
dx = lx / nx
dy = ly / ny

xs = np.linspace(0, lx, nx)
ys = np.linspace(0, ly, ny)
x, y = np.meshgrid(xs, ys)

f = np.exp(-((x - 0.5)**2 + (y - 0.5)**2) / 0.05**2)
vx = 1.5
vy = 1.0

dt = 1.0 * dx / np.sqrt(vx**2 + vy**2) / np.sqrt(2.0)

freq = 2
for n in range(nx * 2):
    f = solve_upwind_2d(f, vx, vy, dx, dy, dt)
    if n % freq == 0:
        plt.pcolor(x, y, f)
        plt.colorbar()
        plt.savefig(f'fig{n//freq:03d}.png', dpi=300, bbox_inches='tight')
        plt.close()


In [12]:
nx = 256
lx = 1.0
xs = np.linspace(0, lx, nx)
dx = xs[1] - xs[0]
dt = 1.0 * lx / nx

# f1 = np.exp(-((xs - 0.5) / 0.1)**2)
f1 = np.where((xs < 0.55) & (xs > 0.35), 1.0, 0.0)
f = solve_upwind(f1, 1.0, dx, dt)
# f = f1

freq = 5
for i in range(nx * 2):
    if i % freq == 0:
        plt.plot(xs, f1)
        ax = plt.gca()
        ax.set_xlabel("$x$")
        ax.set_ylabel("$u$")
        ax.set_ylim(-2.0, 3.0)
        ax.set_xlim(0.0, 1.0)
        ax.axhline(0.0, color='r', ls='--')
        plt.savefig("fig%03d.png" % (i/freq))
        plt.close()
    f_prev = f
    f = solve_leapfrog(f, f1, 1.0, dx, dt)
    f1 = f_prev
    # f = solve_lax(f, 1.0, dx, dt)

In [16]:
nx = 256
lx = 1.0
xs = np.linspace(0, lx, nx)
dx = xs[1] - xs[0]
dt = lx / nx * 0.9

# f = np.exp(-((xs - 0.5) / 0.1)**2)
f = np.where((xs < 0.55) & (xs > 0.35), 1.0, 0.0)
# plt.plot(xs, f)

freq = 5
for i in range(nx * 2):
    if i % freq == 0:
        plt.plot(xs, f)
        ax = plt.gca()
        ax.set_xlabel("$x$")
        ax.set_ylabel("$u$")
        ax.set_ylim(-2, 3)
        ax.set_xlim(0.0, 1.0)
        ax.axhline(0.0, color='r', ls='--')
        plt.savefig("fig%03d.png" % (i/freq))
        plt.close()
    f = solve_lax_wendroff(f, 1.0, dx, dt)

In [31]:
!ffmpeg -r 10 -i fig%03d.png -vcodec libx264 -pix_fmt yuv420p -y lax_2d.mp4

ffmpeg version n7.0.2 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 14.2.1 (GCC) 20240910
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-frei0r --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libdav1d --enable-libdrm --enable-libdvdnav --enable-libdvdread --enable-libfreetype --enable-libfribidi --enable-libgsm --enable-libharfbuzz --enable-libiec61883 --enable-libjack --enable-libjxl --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libplacebo --enable-libpulse --enable-librav1e --enable-librsvg --enable-librubberband --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enabl

In [36]:
!rm *.png